In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [11]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []
keywords='anies'

In [12]:
def scrape_links(page_number,keywords):
    url = f"https://www.cnbcindonesia.com/search?query={keywords}&p={page_number}&kanal=&tipe=&date="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [13]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 12 links from page 1
Total Links: 12


In [14]:
results = []
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[1].strip()
                author_text = author_text.split(',')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('ul', {"class": "breadcrumb"})
            if category_elements:
                category_text = category_elements.find_all('li')
                category_text= category_text[2].get_text()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail_text"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"
            
            nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"

            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [15]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [16]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 12


,title,keywords,author,category,date,content,nama_berita,link
0,Cerita di Balik Food Estate Jokowi yang Disera...,anies,rev,Berita Research,28 September 2023 17:10,"Jakarta, CNBC Indonesia - Lumbung pangan atau ...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023092...
1,"Koalisi Gemuk Prabowo Belum Tentu Menang, Ini ...",anies,Redaksi,Berita,28 September 2023 11:10,"Jakarta, CNBC Indonesia - Prabowo Subianto saa...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928094...
2,"Media Asing Sorot Kaesang Jadi Ketua Partai, S...",anies,sef,Berita,27 September 2023 05:14,"Jakarta, CNBC Indonesia - Media asing kini men...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230927050...
3,"Anies Blak-blakan Kritik Pemerintahan Jokowi, ...",anies,Tim Redaksi,Berita,28 September 2023 07:30,"Jakarta, CNBC Indonesia - Bakal calon Presiden...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928070...
4,Cawapres Anies Blak-blakan: Food Estate Jokowi...,anies,Wiji Nur Hayat,Berita,29 September 2023 06:15,"Jakarta, CNBC Indonesia - Bakal Calon Wakil Pr...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928214...
5,Cawapres Anies Tiba-Tiba Waspada dengan Kaesan...,anies,Wiji Nur Hayat,Berita,27 September 2023 06:15,"Jakarta, CNBC Indonesia - Kaesang Pangarep, pu...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230926203...
6,Ini Dia Daftar Sosok Cawapres Ganjar & Prabowo...,anies,Redaksi,Berita,28 September 2023 13:40,"Jakarta, CNBC Indonesia - Ganjar Pranowo, Anie...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928122...
7,Hasil Survei Terbaru Capres 2024: Ganjar Vs An...,anies,Tim Redaksi,Berita,27 September 2023 04:24,"Jakarta, CNBC Indonesia - Lembaga Indo Riset m...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230927042...
8,"Koalisi Politik Semakin Terbentuk, Siapa Jawar...",anies,Tim RIset,Berita Research,28 September 2023 21:48,"Jakarta, CNBC Indonesia - Waktu pemilihan umum...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023092...
9,Simak Survei Terbaru Capres 2024: Prabowo Vs A...,anies,Tim Redaksi,Berita,29 September 2023 04:00,"Jakarta, CNBC Indonesia - Lembaga Saiful Mujan...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928205...


In [17]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 12


,title,keywords,author,category,date,content,nama_berita,link
0,Cerita di Balik Food Estate Jokowi yang Disera...,anies,rev,Berita Research,28 September 2023 17:10,"Jakarta, CNBC Indonesia - Lumbung pangan atau ...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023092...
1,"Koalisi Gemuk Prabowo Belum Tentu Menang, Ini ...",anies,Redaksi,Berita,28 September 2023 11:10,"Jakarta, CNBC Indonesia - Prabowo Subianto saa...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928094...
2,"Media Asing Sorot Kaesang Jadi Ketua Partai, S...",anies,sef,Berita,27 September 2023 05:14,"Jakarta, CNBC Indonesia - Media asing kini men...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230927050...
3,"Anies Blak-blakan Kritik Pemerintahan Jokowi, ...",anies,Tim Redaksi,Berita,28 September 2023 07:30,"Jakarta, CNBC Indonesia - Bakal calon Presiden...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928070...
4,Cawapres Anies Blak-blakan: Food Estate Jokowi...,anies,Wiji Nur Hayat,Berita,29 September 2023 06:15,"Jakarta, CNBC Indonesia - Bakal Calon Wakil Pr...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928214...
5,Cawapres Anies Tiba-Tiba Waspada dengan Kaesan...,anies,Wiji Nur Hayat,Berita,27 September 2023 06:15,"Jakarta, CNBC Indonesia - Kaesang Pangarep, pu...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230926203...
6,Ini Dia Daftar Sosok Cawapres Ganjar & Prabowo...,anies,Redaksi,Berita,28 September 2023 13:40,"Jakarta, CNBC Indonesia - Ganjar Pranowo, Anie...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928122...
7,Hasil Survei Terbaru Capres 2024: Ganjar Vs An...,anies,Tim Redaksi,Berita,27 September 2023 04:24,"Jakarta, CNBC Indonesia - Lembaga Indo Riset m...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230927042...
8,"Koalisi Politik Semakin Terbentuk, Siapa Jawar...",anies,Tim RIset,Berita Research,28 September 2023 21:48,"Jakarta, CNBC Indonesia - Waktu pemilihan umum...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023092...
9,Simak Survei Terbaru Capres 2024: Prabowo Vs A...,anies,Tim Redaksi,Berita,29 September 2023 04:00,"Jakarta, CNBC Indonesia - Lembaga Saiful Mujan...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230928205...


Save File

In [18]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/cnbc_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')